In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from logicpy import *
from logicpy.core import *
from logicpy.builtin import *
from logicpy.predicate import *
from logicpy.data import *
a = _

import time

In [3]:
u, n = Universe().and_namespace()

n.fib[0, 1] = True
n.fib[1, 2] = True
n.fib[a.N, a.Res] = and_(
    a.N > 1,
    a.N1 << a.N - 1,
    a.N2 << a.N - 2,
    n.fib(a.N1, a.Res1),
    n.fib(a.N2, a.Res2),
    a.Res << a.Res1 + a.Res2)

start = time.time()
assert u.simple_query(n.fib(7, a.X))[0]['X'] == 34
print(f"took {time.time() - start}")

took 5.831412315368652
